In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
import zipfile
from sklearn.preprocessing import LabelEncoder
WORD_EMBEDDING_SIZE = 50
NEWS_FT = 'news_words'
ZIP_FILE = 'data/NewsAggregatorDataset.zip'
DIR_TO_EXTRACT = 'data/'


#Extract the Data
zip_file_ref = zipfile.ZipFile(ZIP_FILE, 'r')
zip_file_ref.extractall(DIR_TO_EXTRACT)
zip_file_ref.close()

/Users/i346047/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/i346047/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
news_df = pd.read_csv('data/newsCorpora.csv',delimiter='\t', header=None, 
                      names=['ID','TITLE','URL','PUBLISHER','CATEGORY','STORY','HOSTNAME','TIMESTAMP'])

In [3]:
news_df = news_df.sample(frac=1.0)
news_df.head(5)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
413630,414149,Apple (AAPL) Offers to Replace Some iPhone 5 B...,http://www.streetinsider.com/Corporate+News/Ap...,StreetInsider.com \(subscription\),t,dq_5Sdpqf86yQOMkrjb3njd0zTiDM,www.streetinsider.com,1409011114974
316187,316647,Former Survivor contestant Caleb Bankston dies...,http://www.thestar.com/entertainment/televisio...,Toronto Star,e,dbafGXQ1otLQSpM8Yuf4I5oxqEmIM,www.thestar.com,1403715304756
322964,323424,AAPL: Apple iPod Touch Gets a Price Cut,http://investorplace.com/2014/06/aapl-apple-ip...,Investorplace.com,t,dALkL3dvsbDC8OMeG9_Mqd-cyPTgM,investorplace.com,1403860222027
369972,370432,Birth Control 'Chip' Could Last 16 Years,http://www.newser.com/story/190595/birth-contr...,Newser,m,dEBVuM2VX4OD7EMMvs5FQ_7xDfAoM,www.newser.com,1404849345376
136201,136537,Internal GM Documents Reveal GM May Have Rejec...,http://www.texomashomepage.com/story/d/story/i...,KFDX,b,dgi3Hd0asMvcxgMXss3LYQnCUbkBM,www.texomashomepage.com,1397887449267


In [4]:
average_title_size = int(sum([len(c) for c in news_df.TITLE])/news_df.shape[0])

In [5]:
lencoder = LabelEncoder()
voc_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(average_title_size)
X_transform = voc_processor.fit_transform(news_df.TITLE)
X_transform = np.array(list(X_transform))
y = lencoder.fit_transform(news_df.CATEGORY.values)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_transform, 
                                    y, test_size=0.2, random_state=42)
n_words = len(voc_processor.vocabulary_)
n_classes = len(lencoder.classes_)

In [6]:
voc_dict = voc_processor.vocabulary_._mapping
sorted_vocab = sorted(voc_dict.items(), key = lambda x : x[1])
f = open('/tmp/meta.tsv', 'w')
for val in sorted_vocab:
    f.write(str(val[0]) + "\n")
f.close()

In [7]:
def get_estimator_spec(input_logits, out_lb, train_predict_m):
    preds_cls = tf.argmax(input_logits, 1)
    if train_predict_m == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=train_predict_m,
        predictions={
            'pred_class': preds_cls,
            'pred_prob': tf.nn.softmax(input_logits)
        })
    tr_l = tf.losses.sparse_softmax_cross_entropy(labels=out_lb, logits=input_logits)
    if train_predict_m == tf.estimator.ModeKeys.TRAIN:
        adm_opt = tf.train.AdamOptimizer(learning_rate=0.01)
        tr_op = adm_opt.minimize(tr_l, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(train_predict_m, loss=tr_l, train_op=tr_op)
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=out_lb, predictions=preds_cls)}
    return tf.estimator.EstimatorSpec(train_predict_m, loss=tr_l, train_op=tr_op)

In [12]:
filter_size=3
num_filters=5
def cnn_model_fn(features,labels,mode):
    news_word_vectors = tf.contrib.layers.embed_sequence(features[NEWS_FT], vocab_size=n_words, 
                                                         embed_dim=WORD_EMBEDDING_SIZE)
    news_word_vectors = tf.expand_dims(news_word_vectors, -1)
    filter_shape = [filter_size, WORD_EMBEDDING_SIZE, 1, num_filters]
    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    conv1 = tf.nn.conv2d(news_word_vectors,
            W,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv1")
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, b), name="relu")
    pool1 = tf.nn.max_pool(
            relu1,
            ksize=[1, average_title_size - 3 + 1, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID',
            name="pool1")
    activations1 = tf.contrib.layers.flatten(pool1)
    logits = tf.contrib.layers.fully_connected(activations1,n_classes,activation_fn=None)
    return get_estimator_spec(input_logits=logits, out_lb=labels, train_predict_m=mode)

In [13]:
run_config = tf.contrib.learn.RunConfig()
run_config = run_config.replace(model_dir='/tmp/models/',save_summary_steps=10,log_step_count_steps=10)
classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,config=run_config)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={NEWS_FT: X_train},
      y=y_train,
      batch_size=len(X_train),
      num_epochs=None,
      shuffle=True)
classifier.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120dd8668>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_secs': 600, '_log_step_count_steps': 10, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/models/'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/model.ckpt.
INFO:tensorflow:loss = 1.3861969, step = 1
INFO:tensorflow:global_step/sec: 0.0425082
INFO:tensorflow:global_step/sec: 0.0439728
INFO:tensorflow:Saving checkpoints for 27 into /tmp/models/model.ckpt.
INFO:tensorflow:global_step/sec: 0.042966


KeyboardInterrupt: 

In [15]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={NEWS_FT: X_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
predictions = classifier.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['pred_class'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)
cls_mets = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(cls_mets))
print(metrics.confusion_matrix(y_test,y_predicted))

INFO:tensorflow:Restoring parameters from /tmp/models/model.ckpt-27
Accuracy: 0.903094
[[20990   534   108  1559]
 [  410 29606   142   331]
 [  493  1432  5741  1381]
 [ 1266   369   162 19960]]
